# Hyperpigmentation Detection System

This notebook implements a comprehensive hyperpigmentation detection system using deep learning models. The system processes original facial images and green-annotated hyperpigmentation masks to train various segmentation models.

## Features:
- Multiple preprocessing techniques for facial images
- Green color segmentation for annotation masks
- Multiple model architectures (U-Net, DeepLabV3+, SegNet)
- Comprehensive evaluation metrics
- Real-time inference pipeline


In [ ]:
# Install required packages
!pip install opencv-contrib-python-headless==4.8.0.74
!pip install --upgrade --force-reinstall mediapipe==0.10.5
!pip install tensorflow==2.13.0
!pip install segmentation-models
!pip install albumentations


In [ ]:
# Mount Google Drive (for Colab)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, Conv2DTranspose, UpSampling2D, BatchNormalization, 
    ReLU, Multiply, Input, GlobalAveragePooling2D, Dense, 
    Activation, MaxPooling2D, Dropout, Add, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers, models
from PIL import Image
import matplotlib.pyplot as plt
import mediapipe as mp
import albumentations as A
from sklearn.model_selection import train_test_split
import json
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def detect_face(image):
    """ Detect face using MediaPipe Face Detection """
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.detections:
            detection = results.detections[0]  # Get the first (most confident) detection
            return detection
    return None

def get_face_bbox(image, detection):
    """Extract face bounding box from detection."""
    h, w = image.shape[:2]
    bbox = detection.location_data.relative_bounding_box
    
    # Convert relative coordinates to absolute coordinates
    x = int(bbox.xmin * w)
    y = int(bbox.ymin * h)
    width = int(bbox.width * w)
    height = int(bbox.height * h)
    
    return x, y, width, height


In [ ]:
# Data Preprocessing Functions

def extract_green_mask(image):
    """
    Extract green-colored hyperpigmentation annotations from the image
    """
    # Convert to HSV for better color segmentation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define range for green color in HSV
    # Green in HSV: H=60-120, S=100-255, V=100-255
    lower_green = np.array([40, 50, 50])   # Lower bound for green
    upper_green = np.array([80, 255, 255])  # Upper bound for green
    
    # Create mask for green regions
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    
    # Morphological operations to clean up the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    
    return green_mask

def enhance_image(image):
    """
    Apply various enhancement techniques to improve hyperpigmentation visibility
    """
    # Convert to LAB color space for better contrast enhancement
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    
    # Merge channels and convert back to BGR
    enhanced_lab = cv2.merge([l, a, b])
    enhanced = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    
    # Apply bilateral filter to reduce noise while preserving edges
    filtered = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    return filtered

def crop_face(image, detection):
    """
    Crop image to face region using face detection
    """
    h, w = image.shape[:2]
    x, y, width, height = get_face_bbox(image, detection)
    
    # Add padding around the face
    padding = 20
    x = max(0, x - padding)
    y = max(0, y - padding)
    width = min(w - x, width + 2 * padding)
    height = min(h - y, height + 2 * padding)
    
    # Crop image
    cropped_image = image[y:y+height, x:x+width]
    
    bbox = (x, y, x+width, y+height)
    return cropped_image, bbox


In [ ]:
# Data Augmentation Pipeline

def get_augmentation_pipeline():
    """
    Define augmentation pipeline for training data
    """
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.RandomGamma(gamma_limit=(80, 120), p=0.3),
        A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
        A.Blur(blur_limit=3, p=0.2),
        A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.3),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.3),
    ])

def apply_augmentation(image, mask, augmentation):
    """
    Apply augmentation to image and mask
    """
    augmented = augmentation(image=image, mask=mask)
    return augmented['image'], augmented['mask']


In [ ]:
# Dataset Preparation

IMG_SIZE = (256, 256)

def preprocess_image(image_path, mask_path, n_classes=1, apply_augmentation_flag=False):
    """
    Preprocess image and mask for training
    """
    # Read image
    image = cv2.imread(image_path.numpy().decode('utf-8'))
    if image is None:
        print(f"Error: Could not read image file at path: {image_path}")
        return np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32), \
               np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
    
    # Enhance image
    image = enhance_image(image)
    
    # Read mask
    mask = cv2.imread(mask_path.numpy().decode('utf-8'), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"Error: Could not read mask file at path: {mask_path}")
        return np.zeros((IMG_SIZE[0], IMG_SIZE[1], 3), dtype=np.float32), \
               np.zeros((IMG_SIZE[0], IMG_SIZE[1], 1), dtype=np.float32)
    
    # Resize
    image = cv2.resize(image, IMG_SIZE)
    mask = cv2.resize(mask, IMG_SIZE, interpolation=cv2.INTER_NEAREST)
    
    # Normalize image
    image = image.astype(np.float32) / 255.0
    
    # Process mask
    if n_classes == 1:
        mask = (mask > 127).astype(np.float32)
        mask = mask[..., None]
    else:
        mask = mask.astype(np.uint8)
        mask = mask[..., None]
    
    return image, mask

def tf_preprocess_image(image_path, mask_path, n_classes=1, apply_augmentation_flag=False):
    """
    TensorFlow wrapper for preprocessing
    """
    image, mask = tf.py_function(
        func=preprocess_image,
        inp=[image_path, mask_path, n_classes, apply_augmentation_flag],
        Tout=[tf.float32, tf.float32]
    )
    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    mask.set_shape([IMG_SIZE[0], IMG_SIZE[1], 1])
    
    return image, mask

def get_dataset(image_dir, mask_dir, batch_size=8, n_classes=1, apply_augmentation_flag=False):
    """
    Create TensorFlow dataset
    """
    image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) 
                         if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) 
                        if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    
    # Ensure same number of files
    min_files = min(len(image_files), len(mask_files))
    image_files = image_files[:min_files]
    mask_files = mask_files[:min_files]
    
    dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))
    dataset = dataset.map(
        lambda x, y: tf_preprocess_image(x, y, n_classes, apply_augmentation_flag), 
        num_parallel_calls=tf.data.AUTOTUNE
    )
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return dataset


In [ ]:
# Model Architectures

class AttentionBlock(tf.keras.layers.Layer):
    """
    Attention block for U-Net
    """
    def __init__(self, filters, **kwargs):
        super(AttentionBlock, self).__init__(**kwargs)
        self.filters = filters
        
    def build(self, input_shape):
        self.theta = Conv2D(self.filters, 1, padding='same')
        self.phi = Conv2D(self.filters, 1, padding='same')
        self.psi = Conv2D(1, 1, padding='same', activation='sigmoid')
        
    def call(self, x, g):
        theta_x = self.theta(x)
        phi_g = self.phi(g)
        f = Activation('relu')(Add()([theta_x, phi_g]))
        psi_f = self.psi(f)
        return Multiply()([x, psi_f])

def conv_block(x, filters, kernel_size=3, padding='same'):
    """
    Convolutional block with batch normalization and ReLU
    """
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def spatial_attention(x):
    """
    Spatial attention mechanism
    """
    avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    sa = Conv2D(1, 7, padding='same', activation='sigmoid')(concat)
    return Multiply()([x, sa])

def build_unet_model(input_shape=(256, 256, 3)):
    """
    Build U-Net model with attention mechanisms
    """
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    conv1 = conv_block(conv1, 64)
    pool1 = MaxPooling2D()(conv1)
    
    conv2 = conv_block(pool1, 128)
    conv2 = conv_block(conv2, 128)
    pool2 = MaxPooling2D()(conv2)
    
    conv3 = conv_block(pool2, 256)
    conv3 = conv_block(conv3, 256)
    pool3 = MaxPooling2D()(conv3)
    
    conv4 = conv_block(pool3, 512)
    conv4 = conv_block(conv4, 512)
    pool4 = MaxPooling2D()(conv4)
    
    # Bottleneck
    bneck = conv_block(pool4, 1024)
    bneck = conv_block(bneck, 1024)
    bneck = spatial_attention(bneck)
    
    # Decoder with attention
    up4 = Conv2DTranspose(512, 2, strides=2, padding='same')(bneck)
    attn4 = AttentionBlock(512)(conv4, up4)
    merge4 = Concatenate()([up4, attn4])
    conv5 = conv_block(merge4, 512)
    conv5 = conv_block(conv5, 512)
    
    up3 = Conv2DTranspose(256, 2, strides=2, padding='same')(conv5)
    attn3 = AttentionBlock(256)(conv3, up3)
    merge3 = Concatenate()([up3, attn3])
    conv6 = conv_block(merge3, 256)
    conv6 = conv_block(conv6, 256)
    
    up2 = Conv2DTranspose(128, 2, strides=2, padding='same')(conv6)
    attn2 = AttentionBlock(128)(conv2, up2)
    merge2 = Concatenate()([up2, attn2])
    conv7 = conv_block(merge2, 128)
    conv7 = conv_block(conv7, 128)
    
    up1 = Conv2DTranspose(64, 2, strides=2, padding='same')(conv7)
    attn1 = AttentionBlock(64)(conv1, up1)
    merge1 = Concatenate()([up1, attn1])
    conv8 = conv_block(merge1, 64)
    conv8 = conv_block(conv8, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(conv8)
    
    model = Model(inputs, outputs, name='Attention_UNet')
    return model


In [ ]:
# Additional Model Architectures

def build_deeplabv3_plus(input_shape=(256, 256, 3)):
    """
    Build DeepLabV3+ model
    """
    # This is a simplified version - for full implementation, use segmentation-models library
    base_model = tf.keras.applications.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # ASPP (Atrous Spatial Pyramid Pooling)
    x = base_model.output
    
    # Global average pooling
    gap = GlobalAveragePooling2D()(x)
    gap = Dense(256, activation='relu')(gap)
    gap = tf.keras.layers.Reshape((1, 1, 256))(gap)
    gap = Conv2DTranspose(256, 32, strides=32, padding='same')(gap)
    
    # Atrous convolutions
    atrous1 = Conv2D(256, 1, padding='same', activation='relu')(x)
    atrous2 = Conv2D(256, 3, padding='same', dilation_rate=6, activation='relu')(x)
    atrous3 = Conv2D(256, 3, padding='same', dilation_rate=12, activation='relu')(x)
    atrous4 = Conv2D(256, 3, padding='same', dilation_rate=18, activation='relu')(x)
    
    # Concatenate
    x = Concatenate()([atrous1, atrous2, atrous3, atrous4, gap])
    x = Conv2D(256, 1, padding='same', activation='relu')(x)
    
    # Decoder
    x = Conv2DTranspose(256, 4, strides=4, padding='same')(x)
    
    # Skip connection
    skip = Conv2D(48, 1, padding='same')(base_model.get_layer('conv2_block3_2_relu').output)
    x = Concatenate()([x, skip])
    
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = Conv2DTranspose(256, 4, strides=4, padding='same')(x)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(x)
    
    model = Model(base_model.input, outputs, name='DeepLabV3Plus')
    return model

def build_segnet(input_shape=(256, 256, 3)):
    """
    Build SegNet model
    """
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    conv1 = conv_block(conv1, 64)
    pool1 = MaxPooling2D()(conv1)
    
    conv2 = conv_block(pool1, 128)
    conv2 = conv_block(conv2, 128)
    pool2 = MaxPooling2D()(conv2)
    
    conv3 = conv_block(pool2, 256)
    conv3 = conv_block(conv3, 256)
    pool3 = MaxPooling2D()(conv3)
    
    conv4 = conv_block(pool3, 512)
    conv4 = conv_block(conv4, 512)
    pool4 = MaxPooling2D()(conv4)
    
    # Bottleneck
    conv5 = conv_block(pool4, 1024)
    conv5 = conv_block(conv5, 1024)
    
    # Decoder
    up4 = Conv2DTranspose(512, 2, strides=2, padding='same')(conv5)
    conv6 = conv_block(up4, 512)
    conv6 = conv_block(conv6, 512)
    
    up3 = Conv2DTranspose(256, 2, strides=2, padding='same')(conv6)
    conv7 = conv_block(up3, 256)
    conv7 = conv_block(conv7, 256)
    
    up2 = Conv2DTranspose(128, 2, strides=2, padding='same')(conv7)
    conv8 = conv_block(up2, 128)
    conv8 = conv_block(conv8, 128)
    
    up1 = Conv2DTranspose(64, 2, strides=2, padding='same')(conv8)
    conv9 = conv_block(up1, 64)
    conv9 = conv_block(conv9, 64)
    
    # Output
    outputs = Conv2D(1, 1, activation='sigmoid', padding='same')(conv9)
    
    model = Model(inputs, outputs, name='SegNet')
    return model


In [ ]:
# Loss Functions and Metrics

def dice_loss(y_true, y_pred, smooth=1e-6):
    """
    Dice loss for segmentation
    """
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
    """
    Focal loss for handling class imbalance
    """
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
    alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    
    focal_loss = alpha_t * tf.pow((1 - p_t), gamma) * tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return tf.reduce_mean(focal_loss)

def combined_loss(y_true, y_pred):
    """
    Combined loss function
    """
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    focal = focal_loss(y_true, y_pred)
    return bce + dice + 0.5 * focal

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    """
    Dice coefficient metric
    """
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def iou_metric(y_true, y_pred, smooth=1e-6):
    """
    Intersection over Union metric
    """
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


In [ ]:
# Training Configuration

# Define paths (update these according to your data structure)
ORIGINAL_IMAGES_PATH = "/content/drive/MyDrive/Dataset/Originals"
ANNOTATED_IMAGES_PATH = "/content/drive/MyDrive/Dataset/Hyperpigmentation_annotated"
OUTPUT_IMAGES_PATH = "/content/drive/MyDrive/Dataset/Hyperpigmentation_Cropped_Images"
OUTPUT_MASKS_PATH = "/content/drive/MyDrive/Dataset/Hyperpigmentation_Cropped_Masks"

# Training parameters
BATCH_SIZE = 8
EPOCHS = 100
LEARNING_RATE = 1e-4
IMG_SIZE = (256, 256)
N_CLASSES = 1

# Create output directories
os.makedirs(OUTPUT_IMAGES_PATH, exist_ok=True)
os.makedirs(OUTPUT_MASKS_PATH, exist_ok=True)

print("Configuration set up successfully!")


In [ ]:
# Data Preprocessing Pipeline

def preprocess_dataset(original_path, annotated_path, output_img_path, output_mask_path):
    """
    Preprocess the entire dataset
    """
    print("Starting dataset preprocessing...")
    
    # Process annotated images to extract green masks
    for filename in os.listdir(annotated_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(annotated_path, filename)
            image = cv2.imread(img_path)
            
            if image is None:
                print(f"Could not read image: {filename}")
                continue
            
            # Detect face
            detection = detect_face(image)
            if detection is None:
                print(f"Could not detect face for: {filename}")
                continue
            
            # Crop to face region
            cropped_img, bbox = crop_face(image, detection)
            if cropped_img is None:
                print(f"Could not crop image: {filename}")
                continue
            
            # Extract green mask
            green_mask = extract_green_mask(cropped_img)
            
            # Save cropped mask
            mask_path = os.path.join(output_mask_path, filename)
            cv2.imwrite(mask_path, green_mask)
            
            print(f"Processed mask: {filename}")
    
    # Process original images
    for filename in os.listdir(original_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(original_path, filename)
            image = cv2.imread(img_path)
            
            if image is None:
                print(f"Could not read image: {filename}")
                continue
            
            # Detect face
            detection = detect_face(image)
            if detection is None:
                print(f"Could not detect face for: {filename}")
                continue
            
            # Crop to face region
            cropped_img, bbox = crop_face(image, detection)
            if cropped_img is None:
                print(f"Could not crop image: {filename}")
                continue
            
            # Save cropped image
            img_output_path = os.path.join(output_img_path, filename)
            cv2.imwrite(img_output_path, cropped_img)
            
            print(f"Processed image: {filename}")
    
    print("Dataset preprocessing completed!")

# Uncomment the line below to run preprocessing
# preprocess_dataset(ORIGINAL_IMAGES_PATH, ANNOTATED_IMAGES_PATH, OUTPUT_IMAGES_PATH, OUTPUT_MASKS_PATH)


In [ ]:
# Model Training

def train_model(model_name='unet'):
    """
    Train the selected model
    """
    # Create datasets
    train_dataset = get_dataset(
        OUTPUT_IMAGES_PATH, 
        OUTPUT_MASKS_PATH, 
        batch_size=BATCH_SIZE, 
        n_classes=N_CLASSES,
        apply_augmentation_flag=True
    )
    
    # Split data for validation (assuming you have validation data)
    # For now, we'll use the same data for both training and validation
    val_dataset = train_dataset.take(10)  # Use first 10 batches for validation
    train_dataset = train_dataset.skip(10)  # Use remaining for training
    
    # Build model
    if model_name == 'unet':
        model = build_unet_model(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    elif model_name == 'deeplabv3':
        model = build_deeplabv3_plus(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    elif model_name == 'segnet':
        model = build_segnet(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss=combined_loss,
        metrics=['accuracy', dice_coefficient, iou_metric]
    )
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7),
        tf.keras.callbacks.ModelCheckpoint(
            f'hyperpigmentation_{model_name}_best.h5', 
            monitor='val_loss', 
            save_best_only=True
        )
    ]
    
    # Train model
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history

# Train different models
print("Training U-Net model...")
unet_model, unet_history = train_model('unet')

print("\nTraining DeepLabV3+ model...")
deeplab_model, deeplab_history = train_model('deeplabv3')

print("\nTraining SegNet model...")
segnet_model, segnet_history = train_model('segnet')


In [ ]:
# Model Evaluation and Visualization

def plot_training_history(history, model_name):
    """
    Plot training history
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss
    axes[0, 0].plot(history.history['loss'], label='Training Loss')
    axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title(f'{model_name} - Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    
    # Accuracy
    axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy')
    axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 1].set_title(f'{model_name} - Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    
    # Dice Coefficient
    axes[1, 0].plot(history.history['dice_coefficient'], label='Training Dice')
    axes[1, 0].plot(history.history['val_dice_coefficient'], label='Validation Dice')
    axes[1, 0].set_title(f'{model_name} - Dice Coefficient')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Dice Coefficient')
    axes[1, 0].legend()
    
    # IoU
    axes[1, 1].plot(history.history['iou_metric'], label='Training IoU')
    axes[1, 1].plot(history.history['val_iou_metric'], label='Validation IoU')
    axes[1, 1].set_title(f'{model_name} - IoU')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('IoU')
    axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

def visualize_predictions(model, dataset, num_samples=5):
    """
    Visualize model predictions
    """
    for images, masks in dataset.take(1):
        predictions = model.predict(images)
        
        for i in range(min(num_samples, images.shape[0])):
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))
            
            # Original image
            axes[0].imshow(images[i])
            axes[0].set_title('Original Image')
            axes[0].axis('off')
            
            # Ground truth mask
            axes[1].imshow(masks[i].squeeze(), cmap='gray')
            axes[1].set_title('Ground Truth')
            axes[1].axis('off')
            
            # Prediction
            pred_mask = (predictions[i].squeeze() > 0.5).astype(np.uint8)
            axes[2].imshow(pred_mask, cmap='gray')
            axes[2].set_title('Prediction')
            axes[2].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            if i >= num_samples - 1:
                break

# Plot training histories
plot_training_history(unet_history, 'U-Net')
plot_training_history(deeplab_history, 'DeepLabV3+')
plot_training_history(segnet_history, 'SegNet')

# Visualize predictions
print("U-Net Predictions:")
visualize_predictions(unet_model, val_dataset)

print("DeepLabV3+ Predictions:")
visualize_predictions(deeplab_model, val_dataset)

print("SegNet Predictions:")
visualize_predictions(segnet_model, val_dataset)


In [ ]:
# Model Comparison and Selection

def evaluate_model(model, dataset, model_name):
    """
    Evaluate model performance
    """
    results = model.evaluate(dataset, verbose=0)
    
    metrics = {
        'Model': model_name,
        'Loss': results[0],
        'Accuracy': results[1],
        'Dice Coefficient': results[2],
        'IoU': results[3]
    }
    
    return metrics

# Evaluate all models
models = {
    'U-Net': unet_model,
    'DeepLabV3+': deeplab_model,
    'SegNet': segnet_model
}

results = []
for name, model in models.items():
    metrics = evaluate_model(model, val_dataset, name)
    results.append(metrics)
    print(f"{name} Results:")
    for key, value in metrics.items():
        print(f"  {key}: {value:.4f}")
    print()

# Create comparison table
import pandas as pd
df = pd.DataFrame(results)
print("Model Comparison:")
print(df.to_string(index=False))

# Select best model based on IoU
best_model_name = df.loc[df['IoU'].idxmax(), 'Model']
best_model = models[best_model_name]
print(f"\nBest model: {best_model_name}")
print(f"Best IoU: {df['IoU'].max():.4f}")


In [ ]:
# Inference Pipeline

def predict_hyperpigmentation(image_path, model, threshold=0.5):
    """
    Predict hyperpigmentation on a new image
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None, None, None
    
    # Detect face
    detection = detect_face(image)
    if detection is None:
        print("Could not detect face")
        return None, None, None
    
    # Crop to face region
    cropped_img, bbox = crop_face(image, detection)
    if cropped_img is None:
        print("Could not crop face region")
        return None, None, None
    
    # Preprocess for model
    processed_img = enhance_image(cropped_img)
    processed_img = cv2.resize(processed_img, IMG_SIZE)
    processed_img = processed_img.astype(np.float32) / 255.0
    
    # Predict
    input_tensor = tf.expand_dims(processed_img, 0)
    prediction = model.predict(input_tensor)
    
    # Threshold prediction
    pred_mask = (prediction[0].squeeze() > threshold).astype(np.uint8)
    
    return cropped_img, pred_mask, bbox

def visualize_prediction(image_path, model, threshold=0.5):
    """
    Visualize prediction on a new image
    """
    cropped_img, pred_mask, bbox = predict_hyperpigmentation(image_path, model, threshold)
    
    if cropped_img is None:
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    # Original cropped image
    axes[0].imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Cropped Face Region')
    axes[0].axis('off')
    
    # Prediction overlay
    overlay = cropped_img.copy()
    overlay[pred_mask > 0] = [0, 255, 0]  # Green overlay for hyperpigmentation
    
    axes[1].imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Hyperpigmentation Detection')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate hyperpigmentation percentage
    total_pixels = pred_mask.size
    hyperpigmentation_pixels = np.sum(pred_mask > 0)
    percentage = (hyperpigmentation_pixels / total_pixels) * 100
    
    print(f"Hyperpigmentation coverage: {percentage:.2f}%")
    
    return percentage

# Example usage with best model
print(f"Using {best_model_name} for inference")

# Test on a sample image (update path as needed)
sample_image_path = "/content/drive/MyDrive/Dataset/Test/sample_image.jpg"
# visualize_prediction(sample_image_path, best_model)


In [ ]:
# Save Models

# Save all trained models
unet_model.save('hyperpigmentation_unet_model.h5')
deeplab_model.save('hyperpigmentation_deeplab_model.h5')
segnet_model.save('hyperpigmentation_segnet_model.h5')

# Save best model with additional metadata
best_model.save(f'hyperpigmentation_{best_model_name.lower()}_best_model.h5')

print("All models saved successfully!")
print(f"Best model ({best_model_name}) saved as: hyperpigmentation_{best_model_name.lower()}_best_model.h5")


In [ ]:
# Additional Utility Functions

def batch_predict_hyperpigmentation(image_folder, model, output_folder, threshold=0.5):
    """
    Batch prediction on multiple images
    """
    os.makedirs(output_folder, exist_ok=True)
    
    results = []
    
    for filename in os.listdir(image_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, filename)
            
            try:
                cropped_img, pred_mask, bbox = predict_hyperpigmentation(image_path, model, threshold)
                
                if cropped_img is not None:
                    # Save prediction mask
                    mask_path = os.path.join(output_folder, f"{filename}_mask.png")
                    cv2.imwrite(mask_path, pred_mask * 255)
                    
                    # Calculate hyperpigmentation percentage
                    total_pixels = pred_mask.size
                    hyperpigmentation_pixels = np.sum(pred_mask > 0)
                    percentage = (hyperpigmentation_pixels / total_pixels) * 100
                    
                    results.append({
                        'filename': filename,
                        'hyperpigmentation_percentage': percentage,
                        'bbox': bbox
                    })
                    
                    print(f"Processed: {filename} - {percentage:.2f}% hyperpigmentation")
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    # Save results to CSV
    import pandas as pd
    df_results = pd.DataFrame(results)
    df_results.to_csv(os.path.join(output_folder, 'hyperpigmentation_results.csv'), index=False)
    
    return results

def create_hyperpigmentation_report(image_path, model, threshold=0.5):
    """
    Create a detailed hyperpigmentation analysis report
    """
    cropped_img, pred_mask, bbox = predict_hyperpigmentation(image_path, model, threshold)
    
    if cropped_img is None:
        return None
    
    # Calculate various metrics
    total_pixels = pred_mask.size
    hyperpigmentation_pixels = np.sum(pred_mask > 0)
    percentage = (hyperpigmentation_pixels / total_pixels) * 100
    
    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(pred_mask, connectivity=8)
    
    # Calculate area statistics
    areas = stats[1:, cv2.CC_STAT_AREA]  # Skip background (label 0)
    
    report = {
        'total_area': total_pixels,
        'hyperpigmentation_area': hyperpigmentation_pixels,
        'hyperpigmentation_percentage': percentage,
        'num_regions': num_labels - 1,  # Exclude background
        'largest_region_area': np.max(areas) if len(areas) > 0 else 0,
        'average_region_area': np.mean(areas) if len(areas) > 0 else 0,
        'bbox': bbox
    }
    
    return report

print("Utility functions defined successfully!")


## Summary

This notebook provides a comprehensive hyperpigmentation detection system with the following features:

### 1. **Data Preprocessing**
- Green color segmentation for annotation masks
- **Overall face detection** using MediaPipe Face Detection (no facial region segmentation)
- Image enhancement using CLAHE and bilateral filtering
- Data augmentation pipeline

### 2. **Model Architectures**
- **U-Net with Attention**: Best for detailed segmentation
- **DeepLabV3+**: Good for multi-scale features
- **SegNet**: Efficient encoder-decoder architecture

### 3. **Training Features**
- Combined loss function (BCE + Dice + Focal)
- Multiple evaluation metrics (Dice, IoU, Accuracy)
- Early stopping and learning rate scheduling
- Model checkpointing

### 4. **Inference Pipeline**
- Real-time prediction on new images
- Batch processing capabilities
- Detailed analysis reports
- Visualization tools

### 5. **Usage Instructions**

1. **Setup**: Update the file paths in the configuration section
2. **Preprocessing**: Run the dataset preprocessing pipeline
3. **Training**: Train all three models and compare performance
4. **Evaluation**: Use the best performing model for inference
5. **Inference**: Apply the model to new images for hyperpigmentation detection

### 6. **Model Recommendations**

Based on typical performance:
- **U-Net**: Best overall performance, good for detailed segmentation
- **DeepLabV3+**: Good for handling multi-scale hyperpigmentation
- **SegNet**: Fastest inference, good for real-time applications

### 7. **Key Changes Made**
- **Simplified Face Detection**: Uses MediaPipe Face Detection instead of facial landmark segmentation
- **Overall Face Cropping**: Crops the entire detected face region instead of specific facial areas
- **Faster Processing**: More efficient preprocessing pipeline

The system automatically selects the best model based on IoU score and provides comprehensive evaluation metrics.
